In [2]:
import requests
import os
from dotenv import load_dotenv
from fuzzywuzzy import process
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from datetime import datetime
import pytz
import pandas as pd

In [3]:
load_dotenv()
odoo_url = "https://dev-unitiva.odoo.com/"
username = os.getenv("user")
pw = os.getenv('pw')
db = os.getenv('db')
api_key= os.getenv('openai_key')

llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o',
    temperature=0
)

# Modelli che richiedono employee_id
modelli_con_employee_id = [
    "hr.leave",          # Richieste di ferie
    "hr.attendance",     # Presenze giornaliere
    "hr.contract",       # Contratti dipendenti
    "hr.expense",        # Note spese
    "account.analytic.line",  # Fogli ore legati ai dipendenti
    "hr.payslip",        # Buste paga
    "hr.skill",          # Competenze del dipendente
    "hr.employee.public", # Record dipendenti pubblici (ridotto)
]

# Modelli che richiedono uid
modelli_con_uid = [
    "res.users",         # Dati relativi all'utente
    "calendar.event",    # Eventi in calendario assegnati all'utente
    "mail.activity",     # Attività pianificate dall'utente
    "res.partner",       # Partner e contatti collegati all'utente
    "crm.lead",          # Opportunità di vendita (CRM)
    "helpdesk.ticket",  # Ticket di supporto assegnati
    'project.project',
    'project.task'
]

In [4]:
#METODO PER EFFETTURA AUTENTICAZIONE SU ODOO (OK)
def autenticazione():
    url_auth = f"{odoo_url}/web/session/authenticate"
    payload_auth = {
        "jsonrpc": "2.0",
        "params": {
            "db":db,
            "login": username,
            "password": pw
        }
    }
    #effettua richiesta
    session = requests.Session()
    response_auth = session.post(url_auth, json=payload_auth)
    response_auth.raise_for_status()
    result = response_auth.json()
    try:
        uid = result['result']['uid']
    except Exception as e:
        return "Autenticazione fallita"
    
    return uid,session

In [11]:
modello = 'project.task'
filtro = []

uid,sessione = autenticazione()


filtro.append(["user_ids", "=", uid])
filtro.append(['project_id', '=', 91])


payload_read = {
    "jsonrpc": "2.0",
    "method": "call",
    "params": {
        "model": modello,
        "method": "search_read",
        "args": [filtro],
        "kwargs": {
            "fields": ["name","project_id"]
        }
    },
    "id": 0
}

url_read = f"{odoo_url}/web/dataset/call_kw/{modello}/search_read"
response_read = sessione.post(url_read, json=payload_read)
response_read.raise_for_status()
result_read = response_read.json().get('result', [])
result_read

[{'id': 2076, 'name': 'Analisi', 'project_id': [91, 'ODOO CHATBOT']}]

In [12]:
payload_read

{'jsonrpc': '2.0',
 'method': 'call',
 'params': {'model': 'project.task',
  'method': 'search_read',
  'args': [[['user_ids', '=', 66], ['project_id', '=', 91]]],
  'kwargs': {'fields': ['name', 'project_id']}},
 'id': 0}